In [1]:
import sys
import os

sys.path.append(os.path.abspath("../.."))

from tiny_graph.graph.executable import Graph
from tiny_graph.constants import START, END

## Testing execution plan conversion


In [ ]:
rprint(simple_graph._convert_execution_plan())

## Testing execution


In [16]:
from datetime import datetime

egraph = Graph()


@egraph.node()
def a():
    time.sleep(1)
    print("a \n", datetime.now())


@egraph.node()
def b():
    time.sleep(1)
    print("b \n", datetime.now())


@egraph.node()
def c():
    time.sleep(1)
    print("c \n", datetime.now())


@egraph.node()
def d():
    time.sleep(1)
    print("d \n", datetime.now())


@egraph.node()
def e():
    time.sleep(1)
    print("e \n", datetime.now())


@egraph.node()
def f():
    time.sleep(1)
    print("f", datetime.now())


@egraph.node()
def g():
    time.sleep(1)
    print("g \n", datetime.now())


egraph.add_edge(START, "a")
egraph.add_edge("a", "b")
egraph.add_edge("b", "c")
egraph.add_edge("b", "d")
egraph.add_edge("d", "e")
egraph.add_edge("c", "f")
egraph.add_edge("f", "g")
egraph.add_edge("e", "g")
egraph.add_edge("g", END)

egraph.compile()
egraph.visualize()

In [ ]:
egraph.execute()

In [10]:
egraph = Graph()


@egraph.node()
def a():
    time.sleep(1)
    print("a", datetime.now())


@egraph.node()
def b():
    time.sleep(1)
    print("b", datetime.now())


@egraph.node()
def c():
    time.sleep(1)
    print("c", datetime.now())


@egraph.node()
def d():
    time.sleep(1)
    print("d", datetime.now())


@egraph.node()
def e():
    time.sleep(1)
    print("e", datetime.now())


@egraph.node()
def f():
    time.sleep(1)
    print("f", datetime.now())


@egraph.node()
def g():
    time.sleep(1)
    time.sleep(1)
    print("g", datetime.now())


egraph.add_edge(START, "a")
egraph.add_edge("a", "b")
egraph.add_edge("b", "c")
egraph.add_edge("c", "d")
egraph.add_edge("d", "e")
egraph.add_edge("e", "f")
egraph.add_edge("f", "g")
egraph.add_edge("g", END)

egraph.compile()
egraph.visualize()

In [ ]:
egraph.execute()

In [ ]:
from rich import print as rprint

# Create a test graph with a mix of parallel and sequential paths
test_graph = Graph()


@test_graph.node()
def start_task():
    print(f"[{datetime.now().strftime('%H:%M:%S.%f')}] Starting workflow \n")
    time.sleep(1)


@test_graph.node()
def parallel_task_1():
    print(f"[{datetime.now().strftime('%H:%M:%S.%f')}] Starting parallel task 1 \n")
    time.sleep(4)
    print(f"[{datetime.now().strftime('%H:%M:%S.%f')}] Finished parallel task 1 \n")


@test_graph.node()
def parallel_task_2():
    print(f"[{datetime.now().strftime('%H:%M:%S.%f')}] Starting parallel task 2 \n")
    time.sleep(4)
    print(f"[{datetime.now().strftime('%H:%M:%S.%f')}] Finished parallel task 2 \n")


@test_graph.node()
def final_task():
    print(f"[{datetime.now().strftime('%H:%M:%S.%f')}] Running final task \n")
    time.sleep(1)


# Create a workflow with parallel execution
test_graph.add_edge(START, "start_task")
test_graph.add_edge("start_task", "parallel_task_1")
test_graph.add_edge("start_task", "parallel_task_2")
test_graph.add_edge("parallel_task_1", "final_task")
test_graph.add_edge("parallel_task_2", "final_task")
test_graph.add_edge("final_task", END)

# Compile and execute
test_graph.compile()
print("\nExecution Plan:")
rprint(test_graph.execution_plan)
print("\nStarting execution:")
start_time = time.time()
test_graph.execute()
end_time = time.time()
print(f"\nTotal execution time: {end_time - start_time:.2f} seconds")

## Testing state


#### Test 1


In [ ]:
from tiny_graph.models.state import GraphState
import time
from tiny_graph.buffer.factory import History, Incremental


# Define a state model with different buffer types
class TestState(GraphState):
    counter: Incremental[int]  # Will accumulate values
    metrics: History[Dict[str, float]]  # Will keep history of all updates
    current_status: LastValue[str]  # Will only keep last value


# Initialize the graph with state
state = TestState(counter=0, metrics={}, current_status="")

test_graph = Graph(state=state)


# Test Incremental Buffer
@test_graph.node()
def add_to_counter(state):
    time.sleep(0.5)
    return {"counter": 5}


@test_graph.node()
def add_more_to_counter(state):
    time.sleep(0.5)
    return {"counter": 3}


# Test History Buffer
@test_graph.node()
def add_metrics(state):
    time.sleep(0.5)
    return {"metrics": {"accuracy": 0.95, "loss": 0.1}}


@test_graph.node()
def update_metrics(state):
    time.sleep(0.5)
    return {"metrics": {"loss": 0.05, "precision": 0.88}}


# Test LastValue Buffer
@test_graph.node()
def set_status_running(state):
    time.sleep(0.5)
    return {"current_status": "running"}


@test_graph.node()
def set_status_complete(state):
    time.sleep(0.5)
    return {"current_status": "complete"}


# Create the workflow
test_graph.add_edge(START, "add_to_counter")
test_graph.add_edge("add_to_counter", "add_more_to_counter")
test_graph.add_edge("add_more_to_counter", "add_metrics")
test_graph.add_edge("add_metrics", "update_metrics")
test_graph.add_edge("update_metrics", "set_status_running")
test_graph.add_edge("set_status_running", "set_status_complete")
test_graph.add_edge("set_status_complete", END)

# Compile and execute
test_graph.compile()
test_graph.execute(timeout=10)

# Print final state
print("\nFinal State:")
print(f"Counter: {test_graph.state.counter}")  # Should be 8 (5 + 3)
print(
    f"Metrics History: {test_graph.state.metrics}"
)  # Should contain both metric updates
print(f"Current Status: {test_graph.state.current_status}")  # Should be "complete"

In [ ]:
test_graph.visualize()

#### Test 2


In [ ]:
from tiny_graph.models.state import GraphState
from typing import Dict
from tiny_graph.buffer.factory import History, Incremental, LastValue
from tiny_graph.graph.executable import Graph
from tiny_graph.constants import START, END


# Define a state model with different buffer types
class TestState(GraphState):
    counter: Incremental[int]  # Will accumulate values
    metrics: History[Dict[str, float]]  # Will keep history of all updates
    current_status: LastValue[str]  # Will only keep last value


# Initialize the graph with state
state = TestState(counter=0, metrics={}, current_status="")

test_graph = Graph(state=state)


# Define nodes that will run in parallel and update the same state
@test_graph.node()
def increment_counter(state):
    time.sleep(0.5)
    return {"counter": 1}


@test_graph.node()
def decrement_counter(state):
    time.sleep(0.5)
    return {"counter": -1}


@test_graph.node()
def update_status(state):
    time.sleep(0.5)
    return {"current_status": "in_progress"}


# Create the workflow with parallel execution
test_graph.add_edge(START, "increment_counter")
test_graph.add_edge(START, "decrement_counter")
test_graph.add_edge(START, "update_status")
test_graph.add_edge("increment_counter", END)
test_graph.add_edge("decrement_counter", END)
test_graph.add_edge("update_status", END)

# Compile and execute
test_graph.compile()
test_graph.execute(timeout=10)

# Print final state
print("\nFinal State:")
print(
    f"Counter: {test_graph.state.counter}"
)  # Should reflect the net effect of increments and decrements
print(
    f"Metrics History: {test_graph.state.metrics}"
)  # Should be empty as no metrics are updated
print(f"Current Status: {test_graph.state.current_status}")  # Should be "in_progress"

In [ ]:
test_graph.execution_plan

In [ ]:
test_graph.visualize()

In [ ]:
graph = Graph()


@graph.node()
def escape():
    print("Starting workflow")


@graph.node()
def process_data():
    print("Processing data")


@graph.node()
def validate():
    print("Validating results")


@graph.node()
def aa():
    print("Validating results")


@graph.node()
def bb():
    print("Validating results")


@graph.node()
def prep():
    print("Workflow complete")


graph.add_edge(START, "process_data")
graph.add_edge("process_data", "validate")
graph.add_edge("validate", "escape")
graph.add_edge("escape", "prep")
graph.add_edge("validate", "aa")
graph.add_edge("aa", "bb")
graph.add_edge("bb", "prep")
graph.add_edge("prep", END)

graph.compile()
graph.visualize()

In [ ]:
graph.execution_plan

#### Test 3


In [ ]:
from tiny_graph.graph.executable import Graph
from tiny_graph.models.state import GraphState
from tiny_graph.buffer.factory import Incremental, LastValue, History
from typing import Dict
from rich import print as rprint


# Define a state model with buffer types
class ComplexTestState(GraphState):
    counter: Incremental[int]  # Will accumulate values
    status: LastValue[str]  # Will only keep last value
    metrics: History[Dict[str, float]]  # Will keep history of all updates


# Initialize the graph with state
state = ComplexTestState(counter=3, status="", metrics={})
graph = Graph(state=state)


# Define nodes that will run in parallel and update the same state
@graph.node()
def increment_counter(state):
    time.sleep(0.5)
    return {"counter": 2}


@graph.node()
def decrement_counter(state):
    time.sleep(0.5)
    return {"counter": -1}


@graph.node()
def update_status_to_in_progress(state):
    time.sleep(0.5)
    return {"status": "in_progress"}


@graph.node()
def update_status_to_complete(state):
    time.sleep(0.5)
    return {"status": "complete"}


@graph.node()
def add_metrics(state):
    time.sleep(0.5)
    return {"metrics": {"accuracy": 0.9, "loss": 0.1}}


@graph.node()
def update_metrics(state):
    time.sleep(0.5)
    return {"metrics": {"loss": 0.05, "precision": 0.85}}


@graph.node()
def finalize_metrics(state):
    time.sleep(0.5)
    return {"metrics": {"loss": 0.01, "precision": 0.99}}


# Create the workflow with multiple levels of execution
graph.add_edge(START, "increment_counter")
graph.add_edge(START, "decrement_counter")
graph.add_edge(START, "update_status_to_in_progress")
graph.add_edge("increment_counter", "add_metrics")
graph.add_edge("decrement_counter", "add_metrics")
graph.add_edge("add_metrics", "update_metrics")
graph.add_edge("update_metrics", "finalize_metrics")
graph.add_edge("update_status_to_in_progress", "update_status_to_complete")
graph.add_edge("update_status_to_complete", "finalize_metrics")
graph.add_edge("finalize_metrics", END)

# Compile and execute
graph.compile()


graph.start(timeout=10)

rprint(graph.state)

# Assert final state
assert (
    graph.state.counter == 1
), "Counter should reflect net effect of increments and decrements"
assert graph.state.status == "complete", "Status should be 'complete'"
assert len(graph.state.metrics) == 3, "Metrics should contain three updates"

In [ ]:
rprint(state)

In [ ]:
graph.visualize()

In [ ]:
from tiny_graph.graph.executable import Graph
from tiny_graph.models.state import GraphState
from tiny_graph.buffer.factory import Incremental, LastValue, History
from typing import Dict
from rich import print as rprint


# Define a state model with buffer types
class ComplexTestState(GraphState):
    counter: Incremental[int]  # Will accumulate values
    status: LastValue[str]  # Will only keep last value
    metrics: History[Dict[str, float]]  # Will keep history of all updates


# Initialize the graph with state
state = ComplexTestState(counter=3, status="", metrics={"xablau": 2.0})
graph = Graph(state=state)


# Define nodes that will run in parallel and update the same state
@graph.node()
def increment_counter(state):
    time.sleep(0.5)
    return {"counter": 2}


@graph.node()
def decrement_counter(state):
    time.sleep(0.5)
    return {"counter": -1}


@graph.node()
def update_status_to_in_progress(state):
    time.sleep(0.5)
    return {"status": "in_progress"}


@graph.node()
def update_status_to_complete(state):
    time.sleep(0.5)
    return {"status": "complete"}


@graph.node()
def add_metrics(state):
    time.sleep(0.5)
    return {"metrics": {"accuracy": 0.9, "loss": 0.1}}


@graph.node()
def update_metrics(state):
    time.sleep(0.5)
    return {"metrics": {"loss": 0.05, "precision": 0.85}}


@graph.node()
def finalize_metrics(state):
    time.sleep(0.5)
    return {"metrics": {"loss": 0.01, "precision": 0.99}}


# Create the workflow with multiple levels of execution
graph.add_edge(START, "increment_counter")
graph.add_edge(START, "decrement_counter")
graph.add_edge(START, "update_status_to_in_progress")
graph.add_edge("increment_counter", "add_metrics")
graph.add_edge("decrement_counter", "add_metrics")
graph.add_edge("add_metrics", "update_metrics")
graph.add_edge("update_metrics", "finalize_metrics")
graph.add_edge("update_status_to_in_progress", "update_status_to_complete")
graph.add_edge("update_status_to_complete", "finalize_metrics")
graph.add_edge("finalize_metrics", END)

# Compile and execute
graph.compile()

In [ ]:
graph.buffers["counter"].value

In [ ]:
graph.buffers["metrics"].value

#### Test 4


In [ ]:
simple_graph = Graph()


# Define some example actions
@simple_graph.node(interrupt="before")
def escape():
    print("Starting workflow")


@simple_graph.node()
def process_data():
    print("Processing data")


@simple_graph.node()
def validate():
    print("Validating results")


@simple_graph.node()
def aa():
    print("Validating results")


@simple_graph.node()
def bb():
    print("Validating results")


@simple_graph.node()
def dd():
    print("Validating results")


@simple_graph.node()
def cc():
    print("Validating results")


@simple_graph.node()
def hh():
    print("Validating results")


@simple_graph.node()
def prep():
    print("Workflow complete")


# Add edges to create workflow
simple_graph.add_edge(START, "process_data")
simple_graph.add_edge("process_data", "validate")
simple_graph.add_edge("validate", "escape")
simple_graph.add_edge("escape", "dd")
simple_graph.add_edge("escape", "cc")
simple_graph.add_edge("cc", "hh")
simple_graph.add_edge("dd", "hh")
simple_graph.add_edge("hh", "prep")
simple_graph.add_edge("validate", "aa")
simple_graph.add_edge("aa", "bb")
simple_graph.add_edge("bb", "prep")
simple_graph.add_edge("prep", END)

simple_graph.compile()

In [ ]:
simple_graph.visualize()

In [ ]:
simple_graph.execution_plan

In [ ]:
from rich import print as rprint

rprint(simple_graph.execution_plan)

In [ ]:
rprint(simple_graph._convert_execution_plan())

In [ ]:
def slice_execution_plan(plan, metadata):
    def slice_list(lst, meta):
        result = []
        for item in lst:
            if isinstance(item, list):
                # Recursively slice nested lists
                sliced_item = slice_list(item, meta)
                if sliced_item:
                    result.append(sliced_item)
            else:
                # Check metadata for slicing
                if meta.get(item) == "before":
                    break
                result.append(item)
                if meta.get(item) == "after":
                    break
        return result

    return slice_list(plan, metadata)


# Example usage
execution_plan = [
    "process_data",
    "validate",
    [["escape", ["dd", "cc"], "hh"], ["aa", "bb"]],
    "prep",
]
metadata = {
    # 'process_data': 'after',
    # 'validate': 'before',
    # 'escape': 'after',
    "dd": "before",
    # 'cc': 'after',
    # 'hh': 'before',
    # 'aa': 'after',
    # 'bb': 'before',
    # 'prep': 'after'
}

sliced_plan = slice_execution_plan(execution_plan, metadata)
print(sliced_plan)

In [ ]:
execution_plan[2:]

In [ ]:
simple_graph.execute()

In [ ]:
# problem here is that in the case of update_status_to_complete,
# it's not checking if the next node is a convergence point

# it needs to have a function that will check if the next node
# connector. find_convergence_point is not doing enough.

## Testing interruption


### Test 1


In [ ]:
from tiny_graph.models.state import GraphState
from typing import Dict
from tiny_graph.buffer.factory import History, Incremental, LastValue


# Define a state model with different buffer types
class TestState(GraphState):
    counter: Incremental[int]  # Will accumulate values
    metrics: History[Dict[str, float]]  # Will keep history of all updates
    current_status: LastValue[str]  # Will only keep last value


# Initialize the graph with state
state = TestState(counter=0, metrics={}, current_status="")

test_graph = Graph(state=state)


# Test Incremental Buffer
@test_graph.node()
def add_to_counter(state):
    time.sleep(0.5)
    print("add_to_counter")
    return {"counter": 5}


@test_graph.node()
def add_more_to_counter(state):
    print("add_more_to_counter")
    time.sleep(0.5)
    return {"counter": 3}


# Test History Buffer
@test_graph.node(interrupt="before")
def add_metrics(state):
    time.sleep(0.5)
    print("add_metrics")
    return {"metrics": {"accuracy": 0.95, "loss": 0.1}}


@test_graph.node()
def update_metrics(state):
    time.sleep(0.5)
    print("update_metrics")
    return {"metrics": {"loss": 0.05, "precision": 0.88}}


# Test LastValue Buffer
@test_graph.node(interrupt="after")
def set_status_running(state):
    time.sleep(0.5)
    print("set_status_running")
    return {"current_status": "running"}


@test_graph.node()
def set_status_complete(state):
    time.sleep(0.5)
    print("set_status_complete")
    return {"current_status": "complete"}


# Create the workflow
test_graph.add_edge(START, "add_to_counter")
test_graph.add_edge("add_to_counter", "add_more_to_counter")
test_graph.add_edge("add_more_to_counter", "add_metrics")
test_graph.add_edge("add_metrics", "update_metrics")
test_graph.add_edge("update_metrics", "set_status_running")
test_graph.add_edge("set_status_running", "set_status_complete")
test_graph.add_edge("set_status_complete", END)

# Compile and execute
test_graph.compile()
test_graph.visualize()

In [ ]:
test_graph.execute()

In [ ]:
test_graph.resume()

In [ ]:
test_graph.resume()

In [ ]:
test_graph.executed_nodes

In [ ]:
from rich import print as rprint


test_graph._convert_execution_plan()
rprint(test_graph.execution_plan)

### Test 2


In [ ]:
x = escape

x.__metadata__["interrupt"]

In [ ]:
simple_graph = Graph()


# Define some example actions
@simple_graph.node(interrupt="before")
def escape():
    time.sleep(0.5)
    print("escape")


@simple_graph.node()
def process_data():
    time.sleep(1.5)
    print("process_data")


@simple_graph.node()
def validate():
    time.sleep(0.5)
    print("validate")


@simple_graph.node()
def aa():
    time.sleep(1.5)
    print("aa")


@simple_graph.node()
def bb():
    time.sleep(0.5)
    print("bb")


@simple_graph.node()
def dd():
    time.sleep(1.5)
    print("dd")


@simple_graph.node()
def cc():
    time.sleep(1.5)
    print("cc")


@simple_graph.node()
def hh():
    time.sleep(0.5)
    print("hh")


@simple_graph.node()
def prep():
    time.sleep(0.5)
    print("prep")


# Add edges to create workflow
simple_graph.add_edge(START, "process_data")
simple_graph.add_edge("process_data", "validate")
simple_graph.add_edge("validate", "escape")
simple_graph.add_edge("escape", "dd")
simple_graph.add_edge("escape", "cc")
simple_graph.add_edge("cc", "hh")
simple_graph.add_edge("dd", "hh")
simple_graph.add_edge("hh", "prep")
simple_graph.add_edge("validate", "aa")
simple_graph.add_edge("aa", "bb")
simple_graph.add_edge("bb", "prep")
simple_graph.add_edge("prep", END)

simple_graph.compile()

simple_graph.visualize()

In [ ]:
from rich import print as rprint

rprint(simple_graph.execution_path)

In [ ]:
from rich import print as rprint

rprint(simple_graph.detailed_execution_path)

In [ ]:
from rich import print as rprint

simple_graph._convert_execution_plan()
rprint(simple_graph.execution_plan)

In [ ]:
from tiny_graph.graph.executable import ExecutableNode
from typing import Union, List, Tuple
from rich import print as rprint


def add_item_to_obj_store(obj_store: Union[List, Tuple], item):
    if isinstance(obj_store, list):
        obj_store.append(item)
        return obj_store  # Return the modified list
    elif isinstance(obj_store, tuple):
        return obj_store + (item,)  # Already returns the new tuple
    else:
        raise ValueError(f"Unsupported object store type: {type(obj_store)}")


def extract_tasks_from_node(node, tasks=[]):
    """
    Extracts tasks from a node, including nested nodes
    returns lists for sequential execution and tuples for parallel execution
    """
    tasks = [] if node.execution_type == "sequential" else tuple()
    for task in node.task_list:
        if isinstance(task, ExecutableNode):
            if task.execution_type == "sequential":
                tasks = add_item_to_obj_store(tasks, extract_tasks_from_node(task, []))
            else:
                tasks = add_item_to_obj_store(
                    tasks, extract_tasks_from_node(task, tuple())
                )
        else:
            tasks = add_item_to_obj_store(tasks, task)

    return tasks


rprint(extract_tasks_from_node(simple_graph.execution_plan[2]))

In [ ]:
simple_graph.execute()

In [ ]:
simple_graph.resume()

In [ ]:
simple_graph.resume()

### Test 3


In [ ]:
from tiny_graph.graph.executable import Graph
from tiny_graph.models.state import GraphState
from tiny_graph.buffer.factory import Incremental, LastValue, History
from typing import Dict
from rich import print as rprint


# Define a state model with buffer types
class ComplexTestState(GraphState):
    counter: Incremental[int]  # Will accumulate values
    status: LastValue[str]  # Will only keep last value
    metrics: History[Dict[str, float]]  # Will keep history of all updates


# Initialize the graph with state
state = ComplexTestState(counter=0, status="", metrics={})
graph = Graph(state=state)


# Define nodes that will run in parallel and update the same state
@graph.node()
def increment_counter(state):
    time.sleep(0.5)
    print("increment_counter")
    return {"counter": 2}


@graph.node()
def decrement_counter(state):
    time.sleep(0.5)
    print("decrement_counter")
    return {"counter": -1}


@graph.node()
def update_status_to_in_progress(state):
    time.sleep(0.5)
    print("update_status_to_in_progress")
    return {"status": "in_progress"}


@graph.node()
def update_status_to_complete(state):
    time.sleep(0.5)
    print("update_status_to_complete")
    return {"status": "complete"}


@graph.node(interrupt="after")
def add_metrics(state):
    time.sleep(0.5)
    print("add_metrics")
    return {"metrics": {"accuracy": 0.9, "loss": 0.1}}


@graph.node()
def update_metrics(state):
    time.sleep(0.5)
    print("update_metrics")
    return {"metrics": {"loss": 0.05, "precision": 0.85}}


@graph.node(interrupt="before")
def finalize_metrics(state):
    time.sleep(0.5)
    print("finalize_metrics")
    return {"metrics": {"loss": 0.01, "precision": 0.99}}


# Create the workflow with multiple levels of execution
graph.add_edge(START, "increment_counter")
graph.add_edge(START, "decrement_counter")
graph.add_edge(START, "update_status_to_in_progress")
graph.add_edge("increment_counter", "add_metrics")
graph.add_edge("decrement_counter", "add_metrics")
graph.add_edge("add_metrics", "update_metrics")
graph.add_edge("update_metrics", "finalize_metrics")
graph.add_edge("update_status_to_in_progress", "update_status_to_complete")
graph.add_edge("update_status_to_complete", "finalize_metrics")
graph.add_edge("finalize_metrics", END)

# Compile and execute
graph.compile()
graph.visualize()

In [ ]:
from rich import print as rprint

rprint(graph.execution_path)

In [ ]:
from rich import print as rprint

graph._convert_execution_plan()
rprint(graph.execution_plan)

In [ ]:
graph.execute()

In [ ]:
graph.resume()

In [ ]:
graph.resume()

In [ ]:
graph.state

### Test 4


In [ ]:
graph.edges

In [ ]:
from tiny_graph.models.state import GraphState
from tiny_graph.buffer.factory import History


class StateForTestWithHistory(GraphState):
    execution_order: History[str]


state = StateForTestWithHistory(execution_order=[])
graph = Graph(state=state)


@graph.node()
def task1(state):
    return {"execution_order": "task1"}


@graph.node(interrupt="after")
def task2(state):
    return {"execution_order": "task2"}


@graph.node(interrupt="before")
def task3(state):
    return {"execution_order": "task3"}


@graph.node()
def task4(state):
    return {"execution_order": "task4"}


graph.add_edge(START, "task1")
graph.add_edge("task1", "task2")
graph.add_edge("task2", "task3")
graph.add_edge("task3", "task4")
graph.add_edge("task4", END)
graph.compile()

In [ ]:
graph.execute()

In [ ]:
graph.resume()

In [ ]:
graph.state

In [ ]:
[edge.id for edge in graph.edges if edge.end_node == "__start__"]

In [ ]:
graph.detailed_execution_path

## Testing Async Execution


### Test 1 (testing non-async using execute_async)


In [ ]:
from tiny_graph.models.state import GraphState
from typing import Dict
from tiny_graph.buffer.factory import History, Incremental, LastValue


# Define a state model with different buffer types
class TestState(GraphState):
    counter: Incremental[int]  # Will accumulate values
    metrics: History[Dict[str, float]]  # Will keep history of all updates
    current_status: LastValue[str]  # Will only keep last value


# Initialize the graph with state
state = TestState(counter=0, metrics=[], current_status="")

test_graph = Graph(state=state)


# Test Incremental Buffer
@test_graph.node()
def add_to_counter(state):
    time.sleep(0.5)
    print("add_to_counter")
    return {"counter": 5}


@test_graph.node()
def add_more_to_counter(state):
    print("add_more_to_counter")
    time.sleep(0.5)
    return {"counter": 3}


# Test History Buffer
@test_graph.node(interrupt="before")
def add_metrics(state):
    time.sleep(0.5)
    print("add_metrics")
    return {"metrics": {"accuracy": 0.95, "loss": 0.1}}


@test_graph.node()
def update_metrics(state):
    time.sleep(0.5)
    print("update_metrics")
    return {"metrics": {"loss": 0.05, "precision": 0.88}}


# Test LastValue Buffer
@test_graph.node(interrupt="after")
def set_status_running(state):
    time.sleep(0.5)
    print("set_status_running")
    return {"current_status": "running"}


@test_graph.node()
def set_status_complete(state):
    time.sleep(0.5)
    print("set_status_complete")
    return {"current_status": "complete"}


# Create the workflow
test_graph.add_edge(START, "add_to_counter")
test_graph.add_edge("add_to_counter", "add_more_to_counter")
test_graph.add_edge("add_more_to_counter", "add_metrics")
test_graph.add_edge("add_metrics", "update_metrics")
test_graph.add_edge("update_metrics", "set_status_running")
test_graph.add_edge("set_status_running", "set_status_complete")
test_graph.add_edge("set_status_complete", END)

# Compile and execute
test_graph.compile()
test_graph.visualize()

In [ ]:
await test_graph.execute_async()

In [ ]:
test_graph.resume()

In [ ]:
test_graph.resume()

In [ ]:
test_graph.executed_nodes

In [ ]:
from rich import print as rprint


test_graph._convert_execution_plan()
rprint(test_graph.execution_plan)

### Test 2


In [ ]:
simple_graph = Graph()


# Define some example actions
@simple_graph.node()
def step_1():
    time.sleep(1)
    print("start")


@simple_graph.node()
def step_A():
    time.sleep(2)
    print("step_A")


@simple_graph.node()
def step_B():
    time.sleep(2)
    print("step_B")


@simple_graph.node()
def step_2():
    time.sleep(1)
    print("step_2")


# Add edges to create workflow
simple_graph.add_edge(START, "step_1")
simple_graph.add_edge("step_1", "step_A")
simple_graph.add_edge("step_1", "step_B")
simple_graph.add_edge("step_A", "step_2")
simple_graph.add_edge("step_B", "step_2")
simple_graph.add_edge("step_2", END)

simple_graph.compile()

simple_graph.visualize()

In [ ]:
simple_graph.start()

In [ ]:
await simple_graph.execute_async()

In [ ]:
simple_graph.execution_plan

### Test 3


In [2]:
import asyncio

simple_graph = Graph()


# Define some example actions
@simple_graph.node()
async def step_1():
    await asyncio.sleep(1)
    print("start")


@simple_graph.node()
async def step_A():
    await asyncio.sleep(2)
    print("step_A")


@simple_graph.node()
async def step_B():
    await asyncio.sleep(2)
    print("step_B")


@simple_graph.node()
async def step_2():
    await asyncio.sleep(1)
    print("step_2")


# Add edges to create workflow
simple_graph.add_edge(START, "step_1")
simple_graph.add_edge("step_1", "step_A")
simple_graph.add_edge("step_1", "step_B")
simple_graph.add_edge("step_A", "step_2")
simple_graph.add_edge("step_B", "step_2")
simple_graph.add_edge("step_2", END)

simple_graph.compile()

simple_graph.visualize()

DEBUG:graphviz._tools:deprecate positional args: graphviz.backend.piping.pipe(['renderer', 'formatter', 'neato_no_op', 'quiet'])
DEBUG:graphviz._tools:deprecate positional args: graphviz.backend.rendering.render(['renderer', 'formatter', 'neato_no_op', 'quiet'])
DEBUG:graphviz._tools:deprecate positional args: graphviz.backend.unflattening.unflatten(['stagger', 'fanout', 'chain', 'encoding'])
DEBUG:graphviz._tools:deprecate positional args: graphviz.backend.viewing.view(['quiet'])
DEBUG:graphviz._tools:deprecate positional args: graphviz.quoting.quote(['is_html_string', 'is_valid_id', 'dot_keywords', 'endswith_odd_number_of_backslashes', 'escape_unescaped_quotes'])
DEBUG:graphviz._tools:deprecate positional args: graphviz.quoting.a_list(['kwargs', 'attributes'])
DEBUG:graphviz._tools:deprecate positional args: graphviz.quoting.attr_list(['kwargs', 'attributes'])
DEBUG:graphviz._tools:deprecate positional args: graphviz.dot.Dot.clear(['keep_attrs'])
DEBUG:graphviz._tools:deprecate posit

In [3]:
await simple_graph.execute_async()

DEBUG:tiny_graph.graph.executable:Chain status updated to: ChainStatus.RUNNING
DEBUG:tiny_graph.graph.executable:Executing task in node: step_1
DEBUG:tiny_graph.graph.executable:Checkpoint saved after node: step_1
DEBUG:tiny_graph.graph.executable:Executing task in node: step_B
DEBUG:tiny_graph.graph.executable:Executing task in node: step_A


start


DEBUG:tiny_graph.graph.executable:Checkpoint saved after node: group_step_B_step_A
DEBUG:tiny_graph.graph.executable:Executing task in node: step_2


step_B
step_A


DEBUG:tiny_graph.graph.executable:Checkpoint saved after node: step_2


step_2
